In [1]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fer2013/fer2013.csv


In [2]:
import os
print(os.listdir('/kaggle/input/fer2013'))

['fer2013.csv']


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
fer_csv = "/kaggle/input/fer2013/fer2013.csv"
df = pd.read_csv(fer_csv)

In [6]:
class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        emotion = int(self.data.iloc[idx]['emotion'])
        pixels = np.array(self.data.iloc[idx]['pixels'].split(), dtype=np.uint8).reshape(48, 48)
        img = np.stack((pixels,) * 3, axis=-1).astype(np.uint8)  # convert to 3-channel

        if self.transform:
            img = self.transform(img)

        return img, emotion

In [7]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((80, 80)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # rotate ±10 degrees
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_df = df[df['Usage'] == 'Training']
val_df = df[df['Usage'] == 'PublicTest']
test_df = df[df['Usage'] == 'PrivateTest']

train_dataset = FERDataset(train_df, train_transform)
val_dataset = FERDataset(val_df, val_transform)
test_dataset = FERDataset(test_df, val_transform)  

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [8]:
class MERcnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),  # Dropout 25%

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Flatten(),
            nn.Linear(256 * 10 * 10, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),  # Higher dropout in FC layers
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 7)
        )

    def forward(self, x):
        return self.network(x)

In [9]:
model = MERcnn().to(device)

# ========================
# Train the Model
# ========================
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def evaluate(model, val_loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    return correct / total


In [10]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Acc: {acc:.4f}")

Epoch 1/10, Loss: 1.7727, Val Acc: 0.3502
Epoch 2/10, Loss: 1.5940, Val Acc: 0.4556
Epoch 3/10, Loss: 1.4317, Val Acc: 0.5026
Epoch 4/10, Loss: 1.3443, Val Acc: 0.5227
Epoch 5/10, Loss: 1.2857, Val Acc: 0.5447
Epoch 6/10, Loss: 1.2467, Val Acc: 0.5506
Epoch 7/10, Loss: 1.2043, Val Acc: 0.5508
Epoch 8/10, Loss: 1.1850, Val Acc: 0.5634
Epoch 9/10, Loss: 1.1660, Val Acc: 0.5812
Epoch 10/10, Loss: 1.1451, Val Acc: 0.5818


In [11]:
torch.save(model.state_dict(), "MERcnn.pth")
print("Model saved as MERcnn.pth")

Model saved as MERcnn.pth
